In [ ]:
# default_exp nbeats.learner

In [ ]:
# hide
import sys

sys.path.append("..")
import pandas as pd
%load_ext autoreload
%autoreload 2

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# N-Beats Learner

> A basic architecture for time series forecasting.


The approach is based on https://arxiv.org/abs/1905.10437


In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastcore.utils import *
from fastcore.imports import *
from fastai2.basics import *
from fastai2.callback.hook import num_features_model
from fastai2.callback.all import *
from fastai2.torch_core import *
from torch.autograd import Variable
from fastseq.all import *

from fastseq.nbeats.metrics import *

## Learner

In [ ]:
# hide
# from fastai2.basics import *
# from fastseq.all import *

@delegates(NBeatsNet.__init__)
def nbeats_learner(dbunch:TSDataLoaders, output_channels=None, metrics=None,cbs=None, theta=0., b_loss=0., loss_func=None, **kwargs):
    "Build a N-Beats style learner"    
    model = NBeatsNet(
        device = dbunch.train.device,
        horizon = dbunch.train.horizon,
        lookback = dbunch.train.lookback, 
        **kwargs
       )
    
    loss_func = ifnone(loss_func, CombinedLoss(F.mse_loss, smape, ratio = {'smape':.0005}))
    learn = Learner(dbunch, model, loss_func=loss_func, opt_func= Adam, 
                    metrics=L(metrics)+L(mae, smape, F.mse_loss, NBeatsTheta(), NBeatsBackwards()),
                    cbs=L(NBeatsTrainer(theta, b_loss))+L(cbs)
                   )
    learn.lh = (dbunch.train.lookback/dbunch.train.horizon)
    return learn

In [ ]:
# hide
from fastseq.data.external import *
items = dummy_data_generator(50, 10, nrows=100)
data = TSDataLoaders.from_items(items, horizon = 7)
learn = nbeats_learner(data,layers=[10], theta = .1, b_loss=.3, stack_types=("seasonality","trend"),
                       loss_func= CombinedLoss(F.mse_loss, smape, ratio = {'smape':.1}),)

learn.fit(1,.001)

Train:500; Valid: 800; Test 100


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tako/dev/env37/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 135, in _worker_loop
    init_fn(worker_id)
  File "/home/tako/dev/fastai2/fastai2/data/load.py", line 17, in _wif
    set_seed(info.seed)
  File "/home/tako/dev/fastai2/fastai2/torch_core.py", line 123, in set_seed
    try: np.random.seed(s%(2**32-1))
TypeError: 'int' object is not callable


In [ ]:
# hide
learn.n_beats_trainer.theta_means()

,seasonality0_0,seasonality0_1,seasonality0_2,trend1_0,trend1_1,trend1_2
0,-0.110665,0.055990,0.076000,-0.006880,0.003721,0.005748
1,0.115849,0.004087,0.072887,0.002325,0.006381,0.004293
2,0.045265,-0.052080,0.020286,-0.000561,0.004114,-0.001551


In [ ]:
# hide
learn.n_beats_trainer.attention()

,seasonality0_0,seasonality0_1,seasonality0_2,trend1_0,trend1_1,trend1_2
mean,0.53,0.50,0.44,0.42,0.49,0.49
std,0.11,0.04,0.09,0.07,0.07,0.07
